## Introduction
    Opinnion mining is a task that if done correctly may give alot of useful information to a buissness,
    Information that may help the buissness grow.

    some examples of the usfuleness of this task:
    1. what is the public opinion about a political canidadte? what does pepole think about his new plan? 
    2. what does pepole think about the new phone? about it's feutures?

    the first question may help in predicting the outcome of an elections or help the candiadte with future decisions.

    the second question may help the company with future decisions and may help on predicting the next day stock value of the company.

in this notebook we will try to use serval deep learning methods to make opinion mining on movie reviews. 


## Data Loading and  Exploratory Analysis
    before we jump into coding, into preprocessing and modeling,
    let's take a few lines of code to understand our data, what info it holds within, it's form and it's level of preprocessing.


In [ ]:
import os 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
#read the data
data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
#pick at the first 5 rows of the data
data.head()

### first impression
    1.we can see that our dataset have two columns:
      the first is the review and the second is the corresponding sentiment.

    2.we can see that the dataset comes unprocessed (in plain languge , with html tags).

now, let's dive a little deeper!

In [ ]:
data.info()

In [ ]:
data.sentiment.value_counts()

### some new insigths
ok so we made another step in our journey of understanding our dataset!
there are two very importent insights:

    1.we have 50,000 reviews and sentiments, there is not any null objects.
    2.our dataset is balanced, we have 25,000 positive reviews and 25,000 negetive.


## Preprocessing
the next step is preprocessing, as we saw in the EDA section the data is in plain text and with tags.
the computer can not understand english or any other languge,
the only thing that it understands is numbers, so our primary goal is to translate the data into computer-understandable lang
    our steps in this section (not nececerly in this order):
    1. data cleaning: 
       get rid of the html tags, punctutions, numbers and one (repeted) letter words (like aaaaa).
    2. dim reduction: 
       lowercase of the text, decontraction of the words (can't->can not) and stop words removal
       (words like is,are that does not give to much information) 
       we will not remove: no, not and nor becuse this kind of stop words gives us alot of information
       and may  change the meaning of the review for example:
       "this is a bad" and "this is not as bad as it seems" have abslutly different meaning.
    3. dim reduction: word normalization. 
    4. spliting of the data into train-dev and test sets with the following ratio (70,15,15).
    5. numerical-encoding of the words (optional removal of un-freq words) based on the train set only(to avoid data-leakege.
    6. padding of the reviews to maxlen.
    7. numerical-encoding of the sentiments.
    8. building an weight-marrix based on FastText vectors for future init of the embedding layer.


In [ ]:
#importing and installing of packages
!pip install stanza
import os,re,nltk,stanza
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

### 1. data cleaning: get rid of the html tags, punctutions and numbers + start of dim reduction with  decontraction of words and lower-casing.
    we will do this step with the help of our old freind regex.


In [ ]:
#for lets prepare all the functions and tools, in the end will apply them all in sequntioal order on the dataframe
def data_cleaning(rev:str)->str:
    """
    lower-casing, decontraction  and cleaning of html tags for each review

    Parameters
    ----------
    rev : str
        the review that needs to be preprocessed.

    Returns
    -------
    rev : str
        the cleaned review .

    """
    rev = re.sub(re.compile('<.*?>'), "", rev.lower())
    rev = re.sub("'s", " is", rev)
    rev = re.sub("'ve", " have", rev)
    rev = re.sub("n't", " not", rev)
    rev = re.sub("cannot", " can not", rev)
    rev = re.sub("'re", " are", rev)
    rev = re.sub("'d", " would", rev)
    rev = re.sub("'ll", " will", rev)
    rev = re.sub("won\'t", "will not", rev)
    rev = re.sub("can\'t", "can not", rev)
    rev = re.sub("\'t", " not", rev)
    rev = re.sub("\'ve", " have", rev)
    rev = re.sub("\'m", " am", rev)
    rev = re.sub("[^a-z ]+", '', rev.replace('.',' ').replace(',',' '))
    return rev

### 2. dim reduction cont.: stop_words removal and the last part of the data cleaning: removal of words like "aaaa".


In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - {'not', 'nor','no'}
def remove_bad_words(rev:str)->str:
    """
     stop words(except no,not and nor) and single character word removal
        
     Parameters
     ----------
     rev : str
         review to be cleaned.

     Returns
     -------
     rev : str
         review after the cleaning process.

     """
    temp=""
    for word in rev.split():
        if word not in stop_words and len(set(word))>1:
            temp+=word+ " "
    return temp

### 3.dim reduction: word normalization.
    the meaning of "ask" and "asked or "help" and "helped" is the same except the time factor,
    in NLP ofthen we want our model to recotnize different forms of the same verb
    as the verb (or root) itself,
    or in other words: we want to map or reduce word into it's basic form\unit.
    
    it is good for us becuase of main two reasons:
        1. the meaning is the same!(if we do not need the tense factor).
        2. dim reduction, instad of having more then on representor to a word
           with the same meaning we will have exactly one (it may boost the preformance of the model).

    essentially: this is a normalization techniqe.

there are two main approtches to achive this goal:

** Stemming **

    it is a heuristic rule-based(regex) approch and it is the simpler approach. 
    this approach tries to reduce the word into it's stem, the stem may or maynot be the acctual morphological root.

** Lemmatization **

    becuse the stemming approtch based on heuristics, it is suffering from problems like: overstemming and understemming.
    the lemmatization taking different approtch, 
    the process of lemmatizing is map a word into it's lemma or in other words: into its dictionary canonical form.

 we will do Lemmatization with the help of stanfords stanza.

In [ ]:
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors="tokenize, mwt,pos,lemma",tokenize_no_ssplit=True)
def lematize(rev:str)->list:
    """
    lemattization of the review. 
        
    Parameters
    ----------
    rev : str
        the review that we want to lemmatize.

    Returns
    -------
    rev : list
        the review after tokenization and lematization.

    """
    return [ word.lemma for sent in nlp(rev).sentences for word in sent.words]

### 4. split the data into train-dev and test sets (train: 36,152 , dev: 6,375, test: 7,473) 
    we must do the spliting before we build the numerical representaion of the words,
    the reason for this is that we want to avoid Data leakage.

**Data leakage**

    we want to  separate our train set from the test and dev sets,
    so we can check the preformance of our model on outside\real-world data.

    if our train will not completele separated from the others sets we will have a data leakage:
    meaning our sets will not show us how our model really preforms on the outside data.
    the result of data leakage can be: good preformance on all the sets but poor preformance on outside data.

    to avoid this problem  we must build numerical representaion (fit) only 
    of our train set and translate the words of all the sets to numbers based on this representaion.


In [ ]:
def split_into_train_test_dev():
    global data
    temp = {}
    temp['train'], temp['test'] = train_test_split(data, test_size=0.15, random_state=42)
    temp['train'] , temp['dev'] = train_test_split(temp['train'], test_size=0.15, random_state=42)
    data = temp

### 5. numerical-encoding of the words (optional removal of un-freq words) based on the train set only

In [ ]:
class Dictionary:
    """
    a class that represents the dictionary of the text
    the class gives a unique index to every word
    """
    def __init__(self):
        self.word2idx = {"PAD":0}
        self.idx2word = {0:"PAD"}
        self.word2freq = {"PAD":1}
        self.idx = 1
    
    def add_words(self,input):
        """
        Input: word or words 
        InputType: list,tuple or str
        a method to add a word(s),
        the method checks if the word(s) is allredy in the dictinoary
        if not, add it.
        othewith adds one to its freq
        """
        def add():
            if word not in self.word2idx.keys():
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.word2freq[word] = 1
                self.idx+=1
            else:
                self.word2freq[word] += 1
       
        def addSeq():
            nonlocal word
            for word in input:
                add()
        inputType = type(input)
        word = []
        if inputType != list and inputType != str and inputType != tuple:
            raise TypeError ("dict at add_word :the type of the input is not allowed")
        if inputType == "str":
            input = [input]
        addSeq()
            
            
    def __getitem__(self,words):
        try:
            if type(words) == str:
                return self.word2idx[words]
            return [self.word2idx[word] for word in words]

        except KeyError:
            print("The word does not exists in the dict")
            return None
    
    def __setitem__(self,word,id):
        """
        add (word,id) to the dict iff the word does not appear in the dict
        """
        if word not in self.word2idx.keys():
            self.word2idx[word] = id 
            self.word2idx[id] = word 
            self.idx+=1

            
    def __len__(self):
        """
        returns the size of the dictionary
        """
        return len(self.word2idx)

In [ ]:
dictionary =Dictionary()
def fit_on_train(rev:list)->list:
    """
    building dictionary based on the words in the train set
    the final result of this method is a dictionary that holds 
    the following information:
    for each word:
        1. it's uniqe id (for future translation of the words in case of embedding layer)
        2. the freq of each word in the set (for optional filtering of un-freqent words)
 

    Parameters
    ----------
    rev : list
        list of words.

    Returns
    -------
    rev : list
        list of words.

    """
    dictionary.add_words(rev)
    return rev

In [ ]:
def transform(rev:list)->list:
    """
    function to transform a review into sequence 

    Parameters
    ----------
    rev : list
        list of tokens.
 
    Returns
    -------
    list
        returns the transformed review.

    """
    return [dictionary[word] for word in rev if dictionary[word]]

### 8. building an weight-marrix based on FastText vectors for future init of the embedding layer.
#### Word Embedding
    This is a very popular set of models that maps words from the vocab into an numerical vectors. 
    These models help the computer “understand” the semantic and syntactical similarity between words and
    the context of the word regarding to the document.
    The word represented by dense vector; similar words will be represented as similar vectors.
    
#### Word2Vec 
    At the year of 2013, Mikolov a researcher at google labs suggested a new model. This model is one
    of the most popular models up to this day.
    This model uses unsupervised shallow neural network with one hidden layer.
    There are two different model architectures which can be leveraged by Word2Vec to create these
    word embedding representations:

##### Continuous Bag of words (CBoW)
    The input of this architecture is bunch of words (vectors) and the output is one word (as vector).
    As a matter of fact, this architecture tries to understand what the word by looking on its surroundings,
    we can look at this as “tell me who your friends and family and I will tell you who you are”
##### Skip Gram
    This is the opposite of CBoW architecture, in this model the input is one word and the output are the
    surrounding words of this word.
    According to Mikolov those to architectures behave differently, and each one of them has its own
    pros and cons.
    The skip-gram model works better with smaller amount of data and can represent rear words better
    than CBoW. On the other hand, CBoW is faster and can represent better words that appears more
    frequently.

##### FastText 
    This model  firstly introduced by Facebook in the year of 2016 as an extension of Word2Vec.
    The main improvement over Word2Vec is the use of sub- word information meaning, while Word2Vec
    tries to find a unique vector for each word, the FastText tries to find a vector for sequences of chars
    within each word. Each word becomes the sum of the sub-word vectors.
we will use a pre-trained FastText vectors to init the weight-marrix of the embedding layer of our model.
words that does not appear in fastText will be randomally init.

In [ ]:
def build_weight_matrix():
    """
    a function that creates weight_matrix out of FastText pre-trained vectors
    for future use as a weights init of an embedding layer
    if there is no vector for the word that we inizlize it with random vector
        
    Returns
    -------
    weight_matrix : list
    """
    def init_fast_text():
        nonlocal fastText,dim
        print("Loading FastText pre-trained vectors")
        with open('../input/fasttext-wikinews/wiki-news-300d-1M.vec', 'r', encoding='utf-8', newline='\n',errors='ignore') as f:
            _, dim = map(int, f.readline().split())
            fastText = {}
            for line in f:
                tokens = line.rstrip().split(' ')
                fastText[tokens[0]] = np.asarray(tokens[1:], "float32")
    
    def build_matrix():
        nonlocal weights_matrix
        print("starting to build weight matrix for embedding encoding,  based on FastText pre-trained vectors")
        maching_words = 0
        dataset_size = len(dictionary)+1
        weights_matrix = np.zeros(shape=(dataset_size,dim))
        for i,word in enumerate(dictionary.word2idx.keys(),1):
            try:
                save = fastText[word]
                maching_words += 1
            except KeyError:
                save = np.random.uniform(size=(dim,))
            weights_matrix[i] = save
                     
        print("pre-treind words: {} randomaly initilaized: {}".format(maching_words,dataset_size))     
    fastText,dim,weights_matrix=[],0,[]
    init_fast_text()
    build_matrix()
    data['matrix'] = weights_matrix

### tie it all together


In [ ]:
data.review = data.review.apply(data_cleaning)
data.review = data.review.apply(remove_bad_words)
data.review = data.review.apply(lematize)
data.sentiment = data.sentiment.map(lambda sent:{'positive':1,'negative':0}[sent])
split_into_train_test_dev()
data["train"].review.apply(fit_on_train)
max_len = data["train"].review.apply(len).max()
for set in ["train","test","dev"]:     
    data[set].review = data[set].review.apply(transform)
    data[set].review = data[set].review.apply(lambda rev:rev+[0]
                                                      *(max_len-len(rev)) 
                                   if len(rev)<max_len else rev[:max_len])
    data[set] ={"X":np.array(data[set].review.to_list()),"Y":np.array(data[set].sentiment.to_list())}   

In [ ]:
build_weight_matrix()

In [ ]:
#for future use of the preprocessed data , so we will not do the preprocess again and agin on the draft , saving!
import pickle
data['dict'] = dictionary
with open("/kaggle/working/data.pkl","wb") as f:
     pickle.dump(data,f)

In [ ]:
#load the  trained data from the last draft session
import pickle
with open('../input/pretrained-data-from-previous-draft-session/data.pkl',"rb") as f:
     data = pickle.load(f)

## Problem defintion and Modeling

    our dataset contains labeled samples so -> our problem is supervised learning.
    for each sample we want identify to which  category(distinct value) it belongs so -> we got ourselves a classification problem.
    and becouse we have only to categorises it is a binary classification problem.
    
    we do not want to  use here classic techniques such as (logistic regression, svm etc.), 
    we will  use nerual network to do this task, neural networks good in text classification problems
    becuse  they can capture non-liniear relationships.
        

### model1: baseline
    for our baseline we will use a simple embeddings model:
        1. input layer.
        2. embedding layer (maps each word into dense vector in the latent space, close words wiil be closed at the 
        vector space(see word embedding at the top). 
        for now, we will init this layer with the weights_matrix.
        3. Flatten -> flat the data for the Dense layer.
        4. output layer: dense (fc) layer.
           **with sigmoid activation function which gives us a  probability of the sample being in the positive category.
         this model is (give or take) a logistic reggresion model.

In [ ]:
#importing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,Flatten 
from tensorflow.keras.callbacks import ModelCheckpoint
#callback: we will monitor val_acc (because it is a classification problem) and save the best model.
def checkpoint(name:str):
    return ModelCheckpoint(name, monitor='val_binary_accuracy', verbose=1, save_best_only=True,
                                   save_weights_only=False, mode='auto', save_freq="epoch")

In [ ]:
#baseline
model = Sequential([Embedding(len(data['dict'])+1, 300,weights=[data['matrix']],input_length=data['train']['X'].shape[-1],
                              mask_zero=True,name="words_latent_space"),
                    Flatten(name="flat"),
                    Dense(1, activation='sigmoid',name='classifyer')],name="Baseline")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
model.summary()
hist = model.fit(data['train']['X'], data['train']['Y'], batch_size=64, epochs=8,
                                  validation_data=(data['dev']['X'],data['dev']['Y']),
                 callbacks=[checkpoint('/kaggle/working/fc_baseline.h5')])

In [ ]:
#configure ploting function for future use
import matplotlib.pyplot as plt
def plot(hist:dict,y_title:str,plot_title:str):
    plt.figure(figsize=(6,2),dpi=140,facecolor="w")
    plt.grid(c='black',linestyle="-",linewidth=2)
    plt.ylabel(y_title)
    plt.xlabel("epochs")
    plt.title(plot_title)
    plt.plot(hist)

In [ ]:
plot(hist.history["binary_accuracy"],"acc","train");plot(hist.history["val_binary_accuracy"],"val_acc","dev")

### insights
   as we can see, our best validation score is  0.8827 (not bad) but: **we have overfiting!**

  
#### Overfiting!
    overfiting ocuurs when the model starts to memorize instead of learning or understanding,
    in this scenario the model will give us a good results on the train set 
    but very bad on dev/test (on un-seen samples). 
    this situation is similar to a student that learns to an exam by memorizing two past exams,
    well sure if he gets by some miracle one of those past exams (aka. data leakage) he will succeed,
    but if he will get a completely new exam, he will fail.
    
    overfiting ussally occurs when our model is too complex for the data.
      
  we can handle overfiting with the following tequnices:
    1. early stopping. we do not need this becuase we are saving the best model.
    2. regularization: intituvly give pennalty to noise
       (or to something that allmost all the time irelvent and relvant only for some minor scenarios).
    3. dropout (also one of the regularization technique:
       randomaly remove a set of connections between the neurons (disable features,make neurons not to fire).
    4. reducing the nn’s capacity(reduce trainable params):
       too many params can add noise, to little can miss importent data.
    5. add data.   
       
most of our network is simple as is, let's try to neglect FastText and let the network try and learn new word embeddings from scratch on smaller latent space.

### Model2: fc+emneddings (less units in latent space)

In [ ]:
model = Sequential([Embedding(len(data['dict'])+1,200,input_length=data['train']['X'].shape[-1],
                              mask_zero=True,name="words_latent_space"),
                    Flatten(name="flat"),
                    Dense(1, activation='sigmoid',name='classifyer')],name="200_emd")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
model.summary()
hist = model.fit(data['train']['X'], data['train']['Y'], batch_size=64, epochs=8,
                                  validation_data=(data['dev']['X'],data['dev']['Y']),
                 callbacks=[checkpoint('/kaggle/working/fc_baseline_200_emd.h5')])

In [ ]:
plot(hist.history["binary_accuracy"],"acc","train");plot(hist.history["val_binary_accuracy"],"val_acc","dev")

### let's add L2 regularizetion on the embeddings weights

In [ ]:
from tensorflow.keras import regularizers

model = Sequential([Embedding(len(data['dict'])+1,200,input_length=data['train']['X'].shape[-1],
                              embeddings_regularizer=regularizers.l2(1e-6),
                              mask_zero=True,name="words_latent_space"),
                    Flatten(name="flat"),
                    Dense(1, activation='sigmoid',name='classifyer')],name="200_emb_d")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
model.summary()
hist = model.fit(data['train']['X'], data['train']['Y'], batch_size=64, epochs=5,
                                  validation_data=(data['dev']['X'],data['dev']['Y']),
                 callbacks=[checkpoint('/kaggle/working/fc_baseline_200_emb_r.h5')])

In [ ]:
plot(hist.history["binary_accuracy"],"acc","train");plot(hist.history["val_binary_accuracy"],"val_acc","dev")

In [ ]:
model = Sequential([Embedding(len(data['dict'])+1,128,input_length=data['train']['X'].shape[-1],
                              embeddings_regularizer=regularizers.l2(1e-6),
                              mask_zero=True,name="words_latent_space"),
                    Flatten(name="flat"),
                    Dense(1, activation='sigmoid',name='classifyer')],name="128_emd")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
model.summary()
hist = model.fit(data['train']['X'], data['train']['Y'], batch_size=64, epochs=5,
                                  validation_data=(data['dev']['X'],data['dev']['Y']),
                 callbacks=[checkpoint('/kaggle/working/128_emd.h5')])

In [ ]:
plot(hist.history["binary_accuracy"],"acc","train");plot(hist.history["val_binary_accuracy"],"val_acc","dev")

## LSTMS
truth be told, the dataset is to small for this, but let give it a shot.
let's add to our baseline a bidirectional lstm with 300 cells
    1. LSTM good for text and sequentes becuse of its memory cells.
    2. bidirectional feed the model from the past to the future as long as future to the past.

In [ ]:
#baseline
from tensorflow.keras.layers import Bidirectional,LSTM
model = Sequential([Embedding(len(data['dict'])+1, 300,weights=[data['matrix']], mask_zero=True),
                    Bidirectional(LSTM(300),name="bi-lstm"),
                    Dense(1, activation='sigmoid',name='classifyer')],name="lstm_base")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
model.summary()
hist = model.fit(data['train']['X'], data['train']['Y'], batch_size=64, epochs=5,
                                  validation_data=(data['dev']['X'],data['dev']['Y']),
                 callbacks=[checkpoint('/kaggle/working/lstm_base.h5')])

In [ ]:
plot(hist.history["binary_accuracy"],"acc","train");plot(hist.history["val_binary_accuracy"],"val_acc","dev")

### Model 2: less cells

In [ ]:
model = Sequential([Embedding(len(data['dict'])+1, 128,mask_zero=True),
                    Bidirectional(LSTM(128),name="bi-lstm"),
                    Dense(1, activation='sigmoid',name='classifyer')],name="lstm_128")

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
model.summary()
hist = model.fit(data['train']['X'], data['train']['Y'], batch_size=64, epochs=5,
                                  validation_data=(data['dev']['X'],data['dev']['Y']),
                 callbacks=[checkpoint('/kaggle/working/lstm_128.h5')])

In [ ]:
plot(hist.history["binary_accuracy"],"acc","train");plot(hist.history["val_binary_accuracy"],"val_acc","dev")

### TEST

In [ ]:
from tensorflow.keras.models import load_model
best_model= load_model('/kaggle/working/lstm_128.h5')
eval=best_model.predict_classes(data['test']['X'])
from sklearn.metrics import classification_report
print(classification_report(eval, data['test']['Y']))                        

### 90 acc and f1 on the test!  not bad!

### Future improvment ideas
there are some improvements that we can try:
    1.  add data, add data, add data, data!!!
    2.  use FastText without training option-> use the weights as is.
    3.  try different pre-trained vectors (Glove or Word2Vec).
    4.  go deeper alongside with better regularization.
    5.  better data cleaning.
    6.  for noise reduction trunscate reviews (instaed of max padding), we need to learn the distrabution of the reviews
        and if there is not so much max_len reviews,we should trunscate them.
    7.  learn the distrabution of words and trunscate the vocab for noise reduction:
        remove un-frequent words (I have made a prepration for this in the dictionary class,
        all we need to do is modify the transform  function.
    8.  do a grid search do determinate the best hyper-paramters including: num of epochs,batch size and learning rate.
    9.  try different optimizers.
    10. for better feture extracion use a conv layer combined with the lstm.
    11. for better capturing of local context along with global context , 
        we can try and combine multi-channel cnn with the original input as input of the lstm.
    12. use self-attention models for sentence level embeddings (BERT,RobeRta, etc.).
 ## first steps should be better preprocessing(4-7) and more data  